# Obtención de datos

In [1]:
import pandas as pd
import regex
import requests
import unidecode
import os, glob
from pathlib import Path  
import cv2
import plotly.express as px
import matplotlib.pyplot as plt 
from tqdm import tqdm
import shutil, sys
import urllib.request
from wordcloud import WordCloud
from sklearn.preprocessing import LabelEncoder
import nltk
from nltk import FreqDist
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/adelaidazuluaga/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/adelaidazuluaga/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Constants

In [2]:
# descargar un total de 1000 datos por clase: 80% train, 20% test
#OFFSETS_TRAIN = list(range(0,550,50)) #550 datos
OFFSETS_LIST= list(range(0,50,50)) #550 datos
#OFFSETS_TEST = list(range(550,700,50)) #150 datos
#OFFSETS_VAL = list(range(700,850,50)) #150 datos

### Requried Installations

In [3]:
from sklearn import metrics


In [4]:
!pip install opencv-python
!pip install plotly==5.10.0
!pip install pillow
!pip install regex
!pip install Unidecode
!pip install wordcloud
!pip install nltk

### Product categories extraction

In [3]:
cats = requests.get('https://api.mercadolibre.com/sites/MLA/categories')
prod_categories= cats.json()

### Obtain product sub categories

In [4]:
def obtain_sub_categories(general_category): 
    try:
        url = f'https://api.mercadolibre.com/sites/MLA/search?category={general_category}'
        request = requests.get(url)
        items = request.json()
        sub_categories_names = [i['name'] for i in items['available_filters'][0]['values']]
        sub_categories_ids = [i['id'] for i in items['available_filters'][0]['values']]
        df_sub_cat = pd.DataFrame(list(zip(sub_categories_ids, sub_categories_names)),columns=['id','Name'])
    except Exception as e:
        print (e.message, e.args)
    return df_sub_cat

### Iteratively download product information for each category

In [26]:
def obtain_json_items_iteratively(df_sub_cat,offset_list):
    offsets= list(range(0,50,50)) #MIL ELEMENTOS POR SUBCATEGORIA
    results = []
    sub_cat_list = []
    offsets = offset_list
    sub_cat_id = list(df_sub_cat['id'])
    sub_cat_name = list(df_sub_cat['Name'])
    for c in sub_cat_id:
        for off in offsets: 
            url = f'https://api.mercadolibre.com/sites/MLA/search?category={c}&offset={off}'
            request = requests.get(url)
            data = request.json()
            try:
                for element in data['results']:
                    results.append(element)
                    sub_cat_list.append(list(df_sub_cat[df_sub_cat['id']==c]['Name'])[0])
            except: 
                print("ERROR",data)
    return results,sub_cat_list

### Extract product brand 

In [6]:
def get_product_brand(items_cat):
    brands = []
    price = []
    for att in items_cat['attributes']:
        find_attr = att['id']
        if 'BRAND' in find_attr or 'Brand' in find_attr:
            brands.append(att['value_name'])
            break
    return brands

### Text cleaning (NLP preprocessing) 

In [7]:
def pre_proc_nlp(text_df):
    cleaned_df =[]
    for text in text_df:
        clean_text = regex.sub(r'\&[a-z]+\;', '', text)
        clean_text = regex.sub(r"\n", " ", text)
        clean_text = text.lower()
        clean_text = unidecode.unidecode(clean_text)
        clean_text = regex.sub(r'\s+', ' ', clean_text)
        clean_text = regex.sub('[^a-zA-Z]', ' ', clean_text).strip(' ')
        cleaned_df.append(clean_text)
    return cleaned_df

In [8]:
def clean_name(text_name):
    text = text_name
    clean_text = regex.sub(r'\&[a-z]+\;', '', text)
    clean_text = regex.sub(r"\n", " ", text)
    clean_text = text.lower()
    clean_text = unidecode.unidecode(clean_text)
    clean_text = regex.sub(r'\s+', ' ', clean_text)
    clean_text = regex.sub('[^a-zA-Z]', ' ', clean_text).strip(' ')
    cleaned_df = clean_text
    return cleaned_df

### Create dataframe for each subset of information

In [9]:
def create_pd_product(item_json,item_category, item_sub_cat):
    titles_=[]
    marcas_=[]
    image_=[]
    price_=[]
    sub_cat_ = []
    items = item_json
    for i in range(len(items)):
        titles_.append(items[i]['title'].lower()) 
        marcas_.append(get_product_brand(items[i])) #extraer la marca
        image_.append(items[i]['thumbnail'])
        price_.append(items[i]['price'])
        sub_cat_.append(clean_name(item_sub_cat[i]))
    df_products = pd.DataFrame(columns=['nombre','categoria','sub_categoria','marca','precio','imagen','nombre_preproc'])
    df_products['nombre'] = titles_
    df_products['categoria'] = clean_name(item_category)
    #df_products['num_categ'] = dict_categs[item_category]*len(items)
    df_products['sub_categoria'] = sub_cat_
    df_products['marca'] = marcas_
    df_products['precio'] = price_
    df_products['imagen'] = image_
    df_products['nombre_preproc'] = pre_proc_nlp(df_products['nombre'])
    return df_products

In [ ]:
#category_dict_train = {}
#category_dict_test = {}
#category_dict_val = {}
#for category in tqdm(prod_categories): 
    #print(category)
    #obtener diccionario de elementos para train, test y val
    #df_cat = obtain_sub_categories(category['id'])
    #item_json_train,item_sub_category_train = obtain_json_items_iteratively(df_cat,OFFSETS_TRAIN) #train
    #item_json_test,item_sub_category_test = obtain_json_items_iteratively(df_cat,OFFSETS_TEST) #test
    #item_json_val,item_sub_category_val = obtain_json_items_iteratively(df_cat,OFFSETS_VAL) #test
    #cat_name = category['name']
    #train dataset 
    #df_cat_prod_train = create_pd_product(item_json_train,cat_name,item_sub_category_train)
    #category_dict_train[df_cat_prod_train['categoria'][0]] = df_cat_prod_train
    #test dataset
    #df_cat_prod_test = create_pd_product(item_json_test,cat_name, item_sub_category_test)
    #category_dict_test[df_cat_prod_test['categoria'][0]] = df_cat_prod_test
    #val dataset
    #df_cat_prod_val = create_pd_product(item_json_val,cat_name, item_sub_category_val)
    #category_dict_val[df_cat_prod_val['categoria'][0]] = df_cat_prod_val

In [ ]:
category_dict = {}
for category in tqdm(prod_categories): 
    #obtener diccionario de elementos para train, test y val
    df_cat = obtain_sub_categories(category['id'])
    item_json,item_sub_category = obtain_json_items_iteratively(df_cat,OFFSETS_LIST) #train
    cat_name = category['name']
    df_cat_prod = create_pd_product(item_json,cat_name,item_sub_category)
    category_dict[df_cat_prod['categoria'][0]] = df_cat_prod

  0%|                                                                                                                                                                              | 0/32 [00:00<?, ?it/s]

In [12]:
df_dataset = pd.concat([category_dict [cat] for cat in category_dict], axis=0)

In [19]:
df_sub_registers = pd.DataFrame(df_dataset['sub_categoria'].value_counts()).reset_index()

In [25]:
df_sub_registers.describe()

,sub_categoria
count,352.000000
mean,54.767045
std,72.507921
min,2.000000
25%,50.000000
50%,50.000000
75%,50.000000
max,1400.000000


* Se decide tomar únicamenteo 50 registros por subcategoria, debido a la distribución de registros en cada subcategoria

### Generar One hot encoding para cada sub categoría

In [15]:
df_dataset_train = pd.concat([category_dict_train[cat] for cat in category_dict_train], axis=0)
df_dataset_test = pd.concat([category_dict_test[cat] for cat in category_dict_test], axis=0)
df_dataset_val = pd.concat([category_dict_val[cat] for cat in category_dict_val], axis=0)

In [16]:
#concatenar los 3 dfs y generar columna de sub clase numerica 
df_dataset_train['subset'] = 'train'
df_dataset_test['subset'] = 'test'
df_dataset_val['subset'] = 'val'
df_total = pd.concat([df_dataset_train,df_dataset_test,df_dataset_val])

In [17]:
def encoding_categs(categories):
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(categories)
    return integer_encoded

In [20]:
numeric_sub_categories = encoding_categs(df_total['sub_categoria'].unique())
dict_sub_categs = dict(zip(df_total['sub_categoria'].unique(),numeric_sub_categories))
num_list = []
for cat_in in df_total['sub_categoria']: 
    num_list.append(dict_sub_categs[cat_in])
df_total['numeric_sub_cat'] = num_list

In [23]:
df_total['numeric_sub_cat'].nunique()

350

In [ ]:
# 350 subcategorías 

In [25]:
df_counts = df_total.groupby('subset').count()

,nombre,categoria,sub_categoria,marca,precio,imagen,nombre_preproc,numeric_sub_cat
subset,,,,,,,,
test,53389,53389,53389,53389,52553,53389,53389,53389
train,202907,202907,202907,202907,198456,202907,202907,202907
val,52301,52301,52301,52301,51614,52301,52301,52301


* Total de registros: 308,597
* Entrenamiento: 202,907 = 65%
* Prueba: 53,389 = 17%
* Validación: 52,301 = 16.9%

# Generate whole dataset 

In [ ]:
cwd = os.getcwd()
filepath = Path(cwd+'/dataset_train_sub_cats.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df_total[df_total['subset']=='train'].to_csv(filepath,index=False)  

filepath = Path(cwd+'/dataset_test_sub_cats.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df_total[df_total['subset']=='test'].to_csv(filepath,index=False)  

filepath = Path(cwd+'/dataset_val_sub_cats.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df_total[df_total['subset']=='val'].to_csv(filepath,index=False)  

In [57]:
df_total

NameError: name 'df_total' is not defined

# Download images per class

In [51]:
def download_images(image_route, category,name_folder):
    for ind,img_url in enumerate(image_route):
        url = img_url
        file_name = category + str(ind) +'.jpg' 
        save_path = './sub_images_'+name_folder+'/'+ category
        # Check whether the specified path exists or not
        if ind==0:
          # Create a new directory because it does not exist 
          os.makedirs(save_path)
        completeName = os.path.join(save_path, file_name)
        if len(url)>0:
            urllib.request.urlretrieve(url, completeName)
        else: 
            print(ind,category)
            print(url)
            print("problema con url")

In [52]:
cwd = os.getcwd()
filepath = Path(cwd+'/dataset_train_sub_cats.csv')  
df_train = pd.read_csv(filepath)

In [53]:
# Download train images
for c_ in tqdm(df_train['categoria'].unique()):
    download_images(df_train[df_train['categoria']==c_]['imagen'],c_,'train')

  0%|                                                                                                                                                                              | 0/32 [20:35<?, ?it/s]

KeyboardInterrupt



In [54]:
ruta = df_train[df_train['categoria']==c_]['imagen']
len(ruta)

11591

In [46]:
%%timeit
for ind,img_url in enumerate(ruta):
    print(ind)
    url = img_url
    file_name = category + str(ind) +'.jpg' 
    save_path = './sub_images_'+'train'+'/'+ 'test'
    # Check whether the specified path exists or not
    isExist = os.path.exists(save_path)
    if ind ==0:
        os.makedirs(save_path)
    completeName = os.path.join(save_path, file_name)
    if len(url)>0:
        urllib.request.urlretrieve(url, completeName)
    else: 
        print(ind,category)
        print(url)
        print("problema con url")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27


KeyboardInterrupt



In [43]:
ind

0

In [ ]:
cwd = os.getcwd()
filepath = Path(cwd+'/dataset_test_sub_cats.csv')  
df_test = pd.read_csv(filepath)

In [ ]:
# Download test images
for c_ in tqdm(df_test['categoria'].unique()):
    download_images(df_test[df_test['categoria']==c_]['imagen'],c_,'test')

In [ ]:
cwd = os.getcwd()
filepath = Path(cwd+'/dataset_val_sub_cats.csv')  
df_val = pd.read_csv(filepath)

In [27]:
# Download test images
for c_ in tqdm(df_val['categoria'].unique()):
    download_images(df_val[df_val['categoria']==c_]['imagen'],c_,'val')